### Mounting drive on Colab

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


###Libraries

In [2]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 29.6 MB/s 


In [3]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Effect of changing hidden layers



In order to study the effects of changing number of hidden layers totl number of neurons must be kept constant across all layers while training the model. Therefore, the following experimental setup is used here:


For Ex: 
        
        If number of neurons be = 1024

        number of hidden layers = 1

        
        If number of hidden layers = 2

        Number of neurons per hidden layer are reduced to 512

        
        Hence, Number of layers: [1, 2, 4, 8, 16, 32, 64]

        Number of Neurons per layer: [1024, 512, 256, 128, 64, 32, 16] 



### For ImageNet as a source

In [8]:
def compile_model(num_neurons, num_layers):
  
  from timeit import default_timer as timer
  class TimingCallback(keras.callbacks.Callback):
      def __init__(self, logs={}):
          self.logs=[]
      def on_epoch_begin(self, epoch, logs={}):
          self.starttime = timer()
      def on_epoch_end(self, epoch, logs={}):
          self.logs.append(timer()-self.starttime)

  cb = TimingCallback()

  from tensorflow.keras.applications.resnet50 import ResNet50
  pre_trained_model = ResNet50(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = None)
#  for layer in pre_trained_model.layers:
#    layer.trainable = False

  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)

  for i in range(num_layers):
    # Add a fully connected layer with "num_neurons" hidden units and ReLU activation
    x = layers.Dense(num_neurons, activation='relu')(x)                  
  
  # Add a final sigmoid layer for classification
  x = layers.Dense  (15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model, cb

In [9]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [10]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [11]:
print("*********** Fitting model for 1024 neurons size and 1 hidden layers")
model, cb = compile_model(1024, 1)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 1024 neurons size and 1 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 77s 273ms/step - loss: 3.4092 - acc: 0.0910 - auc: 0.5477 - recall: 0.0060 - precision: 0.1068 - hamming_loss: 0.9940 - f1_score: 0.0111 - val_loss: 2.7178 - val_acc: 0.0803 - val_auc: 0.5364 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 55s 275ms/step - loss: 2.2721 - acc: 0.2280 - auc: 0.7691 - recall: 0.0424 - precision: 0.6207 - hamming_loss: 0.9576 - f1_score: 0.0731 - val_loss: 2.3651 - val_acc: 0.2190 - val_auc: 0.7520 - val_recall: 0.0123 - val_precision: 0.5522 - val_hamming_loss: 0.9877 - val_f1_score: 0.0232
Epoch 3/10
200/200 [==============================] - 56s 282ms/step - loss: 1.5407 - acc: 0.4555 - auc: 0.9096 - recall: 0.2589 - precision: 0.6895 - hamming_loss: 0.7411 - f1_score: 0.3501 - val_loss: 1.8663 - val_acc: 0.4020 - val_auc: 0.8755 - val_recall: 0.2770 - val_precision: 0.5055 - val_hamming_l

In [12]:
print("*********** Fitting model for 512 dataset size and 2 hidden layers")
model, cb = compile_model(512, 2)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 512 dataset size and 2 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 68s 287ms/step - loss: 3.0846 - acc: 0.0848 - auc_1: 0.5367 - recall_1: 0.0051 - precision_1: 0.1065 - hamming_loss: 0.9949 - f1_score: 0.0094 - val_loss: 2.7444 - val_acc: 0.0677 - val_auc_1: 0.5157 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 56s 280ms/step - loss: 2.3697 - acc: 0.2025 - auc_1: 0.7368 - recall_1: 0.0246 - precision_1: 0.5650 - hamming_loss: 0.9754 - f1_score: 0.0439 - val_loss: 2.4233 - val_acc: 0.1807 - val_auc_1: 0.7286 - val_recall_1: 0.0180 - val_precision_1: 0.6835 - val_hamming_loss: 0.9820 - val_f1_score: 0.0327
Epoch 3/10
200/200 [==============================] - 56s 280ms/step - loss: 1.6534 - acc: 0.4239 - auc_1: 0.8941 - recall_1: 0.2107 - precision_1: 0.6880 - hamming_loss: 0.7893 - f1_score: 0.2965 - val_loss: 2.5450 - val_acc: 0.2567 - val_auc_1: 0.7662 - val_recall_1: 0.1123 - val

In [13]:
print("*********** Fitting model for 256 dataset size and 4 hidden layers")
model, cb = compile_model(256, 4)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 256 dataset size and 4 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 67s 283ms/step - loss: 2.7433 - acc: 0.0834 - auc_2: 0.5351 - recall_2: 0.0010 - precision_2: 0.1437 - hamming_loss: 0.9990 - f1_score: 0.0020 - val_loss: 2.7040 - val_acc: 0.0897 - val_auc_2: 0.5256 - val_recall_2: 0.0000e+00 - val_precision_2: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 56s 279ms/step - loss: 2.4390 - acc: 0.1612 - auc_2: 0.7120 - recall_2: 0.0080 - precision_2: 0.6122 - hamming_loss: 0.9920 - f1_score: 0.0148 - val_loss: 2.3899 - val_acc: 0.1687 - val_auc_2: 0.7392 - val_recall_2: 0.0153 - val_precision_2: 0.5823 - val_hamming_loss: 0.9847 - val_f1_score: 0.0238
Epoch 3/10
200/200 [==============================] - 56s 282ms/step - loss: 1.8321 - acc: 0.3491 - auc_2: 0.8665 - recall_2: 0.1208 - precision_2: 0.6852 - hamming_loss: 0.8792 - f1_score: 0.1813 - val_loss: 2.0777 - val_acc: 0.3370 - val_auc_2: 0.8408 - val_recall_2: 0.1860 - val

In [14]:
print("*********** Fitting model for 128 dataset size and 8 hidden layers")
model, cb = compile_model(128, 8)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 128 dataset size and 8 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 68s 289ms/step - loss: 2.7095 - acc: 0.0759 - auc_3: 0.5188 - recall_3: 8.3333e-05 - precision_3: 0.6667 - hamming_loss: 0.9999 - f1_score: 1.6635e-04 - val_loss: 2.7111 - val_acc: 0.0653 - val_auc_3: 0.5013 - val_recall_3: 0.0000e+00 - val_precision_3: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 55s 277ms/step - loss: 2.5645 - acc: 0.1156 - auc_3: 0.6503 - recall_3: 2.5000e-04 - precision_3: 0.4615 - hamming_loss: 0.9998 - f1_score: 4.9628e-04 - val_loss: 2.8911 - val_acc: 0.1213 - val_auc_3: 0.6371 - val_recall_3: 0.0127 - val_precision_3: 0.1222 - val_hamming_loss: 0.9873 - val_f1_score: 0.0099
Epoch 3/10
200/200 [==============================] - 56s 278ms/step - loss: 2.0596 - acc: 0.2618 - auc_3: 0.8238 - recall_3: 0.0401 - precision_3: 0.6867 - hamming_loss: 0.9599 - f1_score: 0.0618 - val_loss: 2.4168 - val_acc: 0.2207 - val_auc_3: 0.7785 - val_recall

In [15]:
print("*********** Fitting model for 64 dataset size and 16 hidden layers")
model, cb = compile_model(64, 16)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 64 dataset size and 16 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 69s 286ms/step - loss: 2.7018 - acc: 0.0731 - auc_4: 0.5086 - recall_4: 9.5833e-04 - precision_4: 0.6970 - hamming_loss: 0.9990 - f1_score: 0.0019 - val_loss: 2.7080 - val_acc: 0.0667 - val_auc_4: 0.5004 - val_recall_4: 0.0000e+00 - val_precision_4: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 55s 276ms/step - loss: 2.6494 - acc: 0.1022 - auc_4: 0.5735 - recall_4: 0.0041 - precision_4: 0.7259 - hamming_loss: 0.9959 - f1_score: 0.0076 - val_loss: 2.8167 - val_acc: 0.0760 - val_auc_4: 0.5342 - val_recall_4: 0.0127 - val_precision_4: 0.1638 - val_hamming_loss: 0.9873 - val_f1_score: 0.0117
Epoch 3/10
200/200 [==============================] - 55s 277ms/step - loss: 2.3115 - acc: 0.1821 - auc_4: 0.7544 - recall_4: 0.0213 - precision_4: 0.6641 - hamming_loss: 0.9787 - f1_score: 0.0363 - val_loss: 2.3503 - val_acc: 0.2003 - val_auc_4: 0.7589 - val_recall_4: 0.0270 -

In [16]:
print("*********** Fitting model for 32 dataset size and 32 hidden layers")
model, cb = compile_model(32, 32)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 32 dataset size and 32 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 71s 287ms/step - loss: 2.7079 - acc: 0.0686 - auc_5: 0.5000 - recall_5: 0.0000e+00 - precision_5: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7081 - val_acc: 0.0667 - val_auc_5: 0.5000 - val_recall_5: 0.0000e+00 - val_precision_5: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 55s 277ms/step - loss: 2.6759 - acc: 0.1007 - auc_5: 0.5673 - recall_5: 4.1667e-05 - precision_5: 0.5000 - hamming_loss: 1.0000 - f1_score: 8.2406e-05 - val_loss: 2.7130 - val_acc: 0.0883 - val_auc_5: 0.5830 - val_recall_5: 0.0057 - val_precision_5: 0.4359 - val_hamming_loss: 0.9943 - val_f1_score: 0.0094
Epoch 3/10
200/200 [==============================] - 56s 278ms/step - loss: 2.5322 - acc: 0.1169 - auc_5: 0.6766 - recall_5: 0.0100 - precision_5: 0.8047 - hamming_loss: 0.9900 - f1_score: 0.0167 - val_loss: 2.5842 - val_acc: 0.1153 - val_auc_5: 0.6739 - val_re

In [17]:
print("*********** Fitting model for 16 dataset size and 64 hidden layers")
model, cb = compile_model(16, 64)
model.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
test_acc = model.evaluate(X_test, y_test)
print("***********test accuracy is", test_acc,"***************")
print("***********train time is", sum(cb.logs),"***************")
print("f1 score is", sum(test_acc[6])/15)

*********** Fitting model for 16 dataset size and 64 hidden layers


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 74s 290ms/step - loss: 2.7081 - acc: 0.0635 - auc_6: 0.5000 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7080 - val_acc: 0.0667 - val_auc_6: 0.5000 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 53s 267ms/step - loss: 2.7081 - acc: 0.0646 - auc_6: 0.5000 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7080 - val_acc: 0.0673 - val_auc_6: 0.5000 - val_recall_6: 0.0000e+00 - val_precision_6: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
200/200 [==============================] - 53s 267ms/step - loss: 2.7081 - acc: 0.0635 - auc_6: 0.5000 - recall_6: 0.0000e+00 - precision_6: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7080 - val_acc: 0.0673 